In [16]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds



In [17]:
df = pd.read_csv('movies_ratings_cleaned.csv')
df


,userId,movieId,rating,timestamp,title,user_rating,genres_cleaned
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),1,Adventure Animation Children Comedy Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),1,Comedy Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),1,Action Crime Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),1,Mystery Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",1,Crime Mystery Thriller
...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),1,Drama Horror Thriller
100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),1,Action Crime Thriller
100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),1,Horror
100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),1,Action Sci-Fi


In [18]:

# التأكد من أن لدينا عمود التقييمات الصالحة فقط
df_ratings = df[['userId', 'movieId', 'rating']]

# إنشاء مصفوفة التفاعل بين المستخدمين والأفلام (User-Item Matrix)
R_df = df_ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

# تحويل المصفوفة إلى مصفوفة Numpy
R = R_df.to_numpy()

# حساب متوسط التقييمات لكل مستخدم وخصمه من التقييمات
mean_user_rating = np.mean(R, axis=1)
R_demeaned = R - mean_user_rating.reshape(-1, 1)

In [19]:
# تطبيق SVD على المصفوفة المعدلة (R_demeaned)
U, sigma, Vt = svds(R_demeaned, k=50)  # اختيار 50 عامل كامن (يمكن تعديل العدد بناءً على التجربة)

# تحويل sigma إلى مصفوفة قطرية
sigma = np.diag(sigma)

# إعادة بناء التقييمات المتوقعة
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + mean_user_rating.reshape(-1, 1)

# التنبؤ بالتقييمات للمصفوفة (User-Item Matrix)
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=R_df.columns)


In [20]:
# دالة للحصول على التوصيات للمستخدم بناءً على التقييمات المتوقعة
def get_collaborative_recommendations(user_id, num_recommendations=10):
    # الحصول على التقييمات المتوقعة للمستخدم المحدد
    user_row = predicted_ratings_df.iloc[user_id - 1, :]  # ملاحظة أن userId يبدأ من 1
    sorted_indices = user_row.argsort()[::-1]  # ترتيب الأفلام حسب التقييمات المتوقعة
    recommended_movie_ids = sorted_indices[:num_recommendations]  # اختيار أعلى N توصيات
    recommended_movie_titles = df[df['movieId'].isin(R_df.columns[recommended_movie_ids])]['title'].unique()
    return recommended_movie_titles

# اختبار: الحصول على توصيات للمستخدم رقم 1
user_id = 1
collaborative_recs = get_collaborative_recommendations(user_id)
print("Collaborative Filtering Recommendations:")
print(collaborative_recs)


Collaborative Filtering Recommendations:
['Seven (a.k.a. Se7en) (1995)' 'Star Wars: Episode IV - A New Hope (1977)'
 'Fargo (1996)' 'Star Wars: Episode V - The Empire Strikes Back (1980)'
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)'
 'Star Wars: Episode VI - Return of the Jedi (1983)'
 'Indiana Jones and the Last Crusade (1989)' 'Saving Private Ryan (1998)'
 'Indiana Jones and the Temple of Doom (1984)' 'American Beauty (1999)']
